# Reading IQ Data

Only care about reading the raw I/Q samples. In the interest of time, other information are neglected since the development time could be overwhelming.

In [ ]:
import os
import glob
import struct

file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')

In [ ]:
class Object(object):
    pass

In [ ]:
fid = open(file, 'rb')

In [ ]:
# PACK data header
fid.seek(0, 0)
tmp = fid.read(32 * 4)

In [ ]:
tmp

In [ ]:
int.from_bytes(tmp[3*4:4*4], byteorder='big', signed=True)

In [ ]:
n = struct.unpack('IIIIIHH', tmp[:24])
pack_header = Object()
pack_header.pack_check_code_1 = n[0]
pack_header.pack_check_code_2 = n[1]
pack_header.pack_check_counter = n[2]
pack_header.num_cpi = n[3]
pack_header.pack_data_size = n[4]
pack_header.division_number = n[5]
pack_header.division_part_number = n[6]

In [ ]:
print(pack_header.num_cpi)
print(pack_header.pack_data_size)

In [ ]:
fid.close()